In [8]:
%load_ext autoreload
%autoreload 2

import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

import inspect


import apogee.tools.read as apread
import apogee.tools.path as apogee_path
from apogee.tools import bitmask
import collections

from apoNN.src.occam import Occam
from apoNN.src.datasets import ApogeeDataset,AspcapDataset
from apoNN.src.utils import get_mask_elem,dump,load
from apoNN.src.plotting import summarize_representation
import apoNN.src.vectors as vector


import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from sklearn.decomposition import PCA

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2,interaction_only=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

apogee_path.change_dr(16)

[(['TEFF', 'LOGG', 'LOG10VDOP', 'METALS', 'C', 'N', 'O Mg Si S Ca Ti'], ['C', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'S', 'K', 'Ca', 'Ti', 'V', 'Mn', 'Fe', 'Ni'], ['[C/M]', '[N/M]', '[O/M]', '[Na/H]', '[Mg/M]', '[Al/H]', '[Si/M]', '[S/M]', '[K/H]', '[Ca/M]', '[Ti/M]', '[V/H]', '[Mn/H]', '[Fe/H]', '[Ni/H]'], [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1])]


### Generating z_ae

z_ae is the raw latents for the dataset

In [9]:
elem = "Fe"
mask_elem  = get_mask_elem(elem)
autoencoder = torch.load("/share/splinter/ddm/taggingProject/apogeeFactory/outputs/guild/window/fel2/ae_20000.p",map_location=device)

/share/splinter/ddm/modules/apogee/apogee/spec/window.py:89: RuntimeWarning: invalid value encountered in greater
  mask= ((win > 0.)*(True^numpy.isnan(win))).astype('int')


In [10]:
allStar =  load("allStar_training_clean")
dataset=  AspcapDataset(filename="aspcap_training_clean",tensor_type=torch.FloatTensor,recenter=True)
n_data=10000

In [11]:
z_ae = vector.LatentVector(dataset,autoencoder,n_data)


/share/splinter/ddm/taggingProject/apogeeFactory/apoNN/src/vectors.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _,z = self.autoencoder(torch.tensor(self.dataset[idx][0]).to(device).unsqueeze(0))


### Filtering z_ae

We want to do linear regression on the set of spectra. We must thus remove all spectra for which the enhancement estimation did not converge (these spectra then have "X_FE"=-9999)

In [12]:
feh_outliercut = allStar["Fe_H"]>-5
o_outliercut = allStar["O_FE"]>-5
c_outliercut = allStar["C_FE"]>-5
na_outliercut = allStar["Na_FE"]>-5
mg_outliercut = allStar["Mg_FE"]>-5
si_outliercut = allStar["Si_FE"]>-5
al_outliercut = allStar["Al_FE"]>-5
s_outliercut = allStar["S_FE"]>-5
p_outliercut = allStar["P_FE"]>-5
ti_outliercut = allStar["Ti_FE"]>-5
cr_outliercut = allStar["Cr_FE"]>-5



combined_cut = feh_outliercut & o_outliercut &  c_outliercut & na_outliercut & mg_outliercut & si_outliercut & al_outliercut & p_outliercut & s_outliercut & ti_outliercut & cr_outliercut

we create a second order polynomial with the features

In [ ]:
considered_parameters = ["Teff","logg","Fe_H","O_FE","C_FE","Na_FE","Mg_FE","Si_FE","S_FE","Al_FE","P_FE","Ni_FE"] 
y = vector.Vector(np.array([allStar[:n_data][combined_cut[:n_data]][param] for param in considered_parameters])[:,:n_data].T)
y_astronn = vector.AstroNNVector(allStar[:n_data][combined_cut[:n_data]],considered_parameters)

In [ ]:
z = vector.Vector(z_ae.raw[combined_cut[:n_data]],order=2,interaction_only=False)

### Carrying out linear regression

We can study the linearity of our resultant representation by carrying linear and non-linear regression with the latents. We use the astroNN abundances as a gold-standard


In [ ]:
w = vector.LinearTransformation(z,y)
nonlinear_w = vector.NonLinearTransformation(z,y)
nonlinear_w.fit(n_epochs=1000)

In [ ]:
for i in range(len(considered_parameters)):
    plt.title(considered_parameters[i])
    nonlinear_std = np.std(nonlinear_w.predict(z).raw[:,i]-y.raw[:,i])
    linear_std = np.std(w.predict(z).raw[:,i]-y.raw[:,i])
    astronn_std = np.std(y_astronn.raw[:,i]-y.raw[:,i])


    plt.scatter(y.raw[:,i],w.predict(z).raw[:,i],s=0.2,label="linear:{:.4f}".format(linear_std))
    plt.scatter(y.raw[:,i],nonlinear_w.predict(z).raw[:,i],s=0.2,label="nonlinear:{:.4f}".format(nonlinear_std))
    plt.scatter(y.raw[:,i],y_astronn.raw[:,i],s=0.2,label="astronn:{:.4f}".format(astronn_std))

    plt.legend()
    plt.show()

In [ ]:
for i in range(len(considered_parameters)):
    plt.title(considered_parameters[i])
    nonlinear_std = np.std(nonlinear_w.predict(z).raw[:,i]-y.raw[:,i])
    linear_std = np.std(w.predict(z).raw[:,i]-y.raw[:,i])
    astronn_std = np.std(y_astronn.raw[:,i]-y.raw[:,i])


    plt.scatter(y.raw[:,i],w.predict(z).raw[:,i],s=0.2,label="linear:{:.4f}".format(linear_std))
    plt.scatter(y.raw[:,i],nonlinear_w.predict(z).raw[:,i],s=0.2,label="nonlinear:{:.4f}".format(nonlinear_std))
    plt.scatter(y.raw[:,i],y_astronn.raw[:,i],s=0.2,label="astronn:{:.4f}".format(astronn_std))

    plt.legend()
    plt.show()

## Occam

### Loading Occam data

In [ ]:
occam = load("occam")
allStar_occam = occam["allStar"]
dataset_occam = AspcapDataset(filename="aspcap_occam",recenter=True,tensor_type=torch.FloatTensor,filling_dataset=dataset.dataset["aspcap"])

In [ ]:
occam_cluster_idxs = occam["cluster_idxs"]
z_occam = vector.OccamLatentVector(occam_cluster_idxs,dataset=dataset_occam,autoencoder = autoencoder,n_data = len(dataset_occam))


In what follows, we show a simple loop for identifying, using the reconstruction loss, any outlier spectra which are not well captured by the autoencoder

In [ ]:
bad_indexes = []
for i in range(len(z_occam.raw)):
    err = np.sqrt(np.sum(((z_occam.get_x(i)[mask_elem.astype(bool)]-z_occam.get_x_pred(i))**2).detach().numpy()))
    print(err)
    if err>0.5:
        print(err)
        print(i)
        bad_indexes.append(i)
        plt.plot(z_occam.get_x(i)[mask_elem.astype(bool)])
        plt.plot(z_occam.get_x_pred(i))
        #z_occam.plot(i)
        plt.show()

We leave a commentedout piece of code for deleting any outlier spectra, if judged necessary.

retained= np.delete(np.arange(len(z_occam.raw)),bad_indexes)
retained

In [ ]:
z_occam = vector.OccamLatentVector(occam_cluster_idxs,raw=z_occam.raw,order=2,interaction_only=False)

### PCA investigation

The PCA procedure involves two steps:

1. Transform the latent representations to a whitened representation. In the whitened representation, all factors of variation in the full dataset have appropriately equal variance. 

2. Find those factors of variations, in the whitened representation, which have lowest intracluster variance. 

In [ ]:
whitener = PCA(n_components=z.raw.shape[1],whiten=True)#z.raw.shape[1],whiten=True)


In [ ]:
whitener.fit(z.centered)

now that we have learned a whitener. We transform our dataset to the new whitened space

In [ ]:
w_z_c = whitener.transform(z.centered)
w_z_occam_c = whitener.transform(z_occam.cluster_centered)

Normally here the variance in all the directions, for the full dataset, should be roughly equal. We can check this

In [ ]:
pca = PCA(n_components=z.raw.shape[1])
pca.fit(w_z_c[:2500,:])
print(pca.explained_variance_)

Hopefully when we look at clusters, there should be some directions along which there are not really any variations

In [ ]:
pca.fit(w_z_occam_c)
print(pca.explained_variance_)

### Assessing Open clusters

Our latent representations merit is evaluated by looking at how much variance is found within clusters compared to within full dataset. Because we don't want our representation to be influenced by outlier datapoints (where the model has failed), we threshold the most outlier datapoints.

**In this exercise, we compare Fe/H, the most well estimated parameter, to our latents smallest direction of variation**

In [6]:
dim = 18
zc_b = vector.project(w_z_c,pca.components_[dim][None,:])
zoccam_b = vector.project(w_z_occam_c,pca.components_[dim][None,:])

NameError: name 'vector' is not defined

In [7]:
summarize_representation(zc_b,zoccam_b,0.2,0.2)

NameError: name 'summarize_representation' is not defined

### Comparing to raw abundances

We can compare our representations expressiveness to the raw abundances.

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
abundances_occam = []
elem = "Fe_H"
elem_idx = considered_parameters.index(elem)

for cluster in z_occam.registry:
    clust_idxs = z_occam.registry[cluster]
    abundances_occam.append(allStar_occam[clust_idxs][elem]-np.mean(allStar_occam[clust_idxs][elem]))
    #print(z_occam.dataset.allStar[clust_idxs][elem]-z_occam.dataset.allStar[clust_idxs][elem])
abundances_occam = np.array(flatten(abundances_occam))

In [ ]:
abundances_all = y.raw[:,elem_idx]-np.mean(y.raw[:,elem_idx])

In [ ]:
summarize_representation(abundances_all,abundances_occam,0.2,0.2)

### Comparing to astroNN

In [3]:
y_astronn = vector.AstroNNVector(allStar[:n_data][combined_cut[:n_data]],considered_parameters)
y_astronn_occam = vector.AstroNNVector(allStar_occam,considered_parameters)

NameError: name 'vector' is not defined

In [4]:
abundances_nn_occam = []
elem = "Fe_H"
elem_idx = considered_parameters.index(elem)

for cluster in z_occam.registry:
    clust_idxs = z_occam.registry[cluster]
    abundances_nn_occam.append(y_astronn_occam.raw[clust_idxs,elem_idx]-np.mean(y_astronn_occam.raw[clust_idxs,elem_idx]))
    #print(z_occam.dataset.allStar[clust_idxs][elem]-z_occam.dataset.allStar[clust_idxs][elem])
abundances_nn_occam = np.array(flatten(abundances_nn_occam))

NameError: name 'considered_parameters' is not defined

In [5]:
abundances_nn_all  =y_astronn.raw[:,elem_idx]-np.mean(y_astronn.raw[:,elem_idx])


NameError: name 'y_astronn' is not defined

In [ ]:
summarize_representation(abundances_nn_all,abundances_nn_occam,0.2,0.2)